In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='secret-scope')]

In [0]:
dbutils.secrets.list("secret-scope")

[SecretMetadata(key='client-id'),
 SecretMetadata(key='demodbricks472-sp'),
 SecretMetadata(key='tenant-id')]

In [0]:
service_credential = dbutils.secrets.get(scope="secret-scope",key="demodbricks472-sp")
app_id = dbutils.secrets.get(scope="secret-scope",key="client-id")
dir_id = dbutils.secrets.get(scope="secret-scope",key="tenant-id")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": app_id,
          "fs.azure.account.oauth2.client.secret": service_credential,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{dir_id}/oauth2/token"}

mount_point = "/mnt/arunkumar1234"

all_mounts = [m.mountPoint for m in dbutils.fs.mounts()]

if mount_point in all_mounts:
    dbutils.fs.unmount(mount_point)
    dbutils.fs.mount(
        source = "abfss://raw@arunkumar1234.dfs.core.windows.net/",
        mount_point = "/mnt/arunkumar1234",
        extra_configs = configs)
else:
    dbutils.fs.mount(
        source = "abfss://raw@arunkumar1234.dfs.core.windows.net/",
        mount_point = "/mnt/arunkumar1234",
        extra_configs = configs)

/mnt/arunkumar1234 has been unmounted.


In [0]:
dbutils.fs.ls("/mnt/arunkumar1234")

[FileInfo(path='dbfs:/mnt/arunkumar1234/2019.csv', name='2019.csv', size=123984, modificationTime=1739473662000),
 FileInfo(path='dbfs:/mnt/arunkumar1234/2020.csv', name='2020.csv', size=283504, modificationTime=1739476248000),
 FileInfo(path='dbfs:/mnt/arunkumar1234/delta/', name='delta/', size=0, modificationTime=1739475338000)]

In [0]:
#df=spark.read.format("csv").option("header", "true").load("/mnt/arunkumar1234/2019.csv")
#df.show(2)

In [0]:
#df.selectExpr("*", 
#              "year(OrderDate) as year", 
#              "month(OrderDate) as month", 
#              "day(OrderDate) as day") \
#  .write.format("delta") \
#  .mode("overwrite") \
#  .partitionBy("year", "month", "day") \
#  .save("/mnt/arunkumar1234/delta")

In [0]:
#%sql
#create database if not exists forecast;

In [0]:
#df.selectExpr("*", 
#              "year(OrderDate) as year", 
#              "month(OrderDate) as month", 
#              "day(OrderDate) as day") \
#  .write.format("delta") \
#  .mode("overwrite") \
#  .partitionBy("year", "month", "day") \
#  .saveAsTable("forecast.sales_data")

In [0]:
df1=spark.read.format("csv").option("header", "true").load("/mnt/arunkumar1234/2020.csv")
df1.show(2)

+----------------+--------------------+----------+-------------+--------------------+------------------+--------+---------+---------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate| CustomerName|        EmailAddress|              Item|Quantity|UnitPrice|TaxAmount|
+----------------+--------------------+----------+-------------+--------------------+------------------+--------+---------+---------+
|         SO45347|                   1|2020-01-01|Clarence Raji|clarence35@advent...|Road-650 Black, 52|       1| 699.0982|  55.9279|
|         SO45345|                   1|2020-01-01|  Bonnie Yuan|bonnie12@adventur...|  Road-150 Red, 52|       1|  3578.27| 286.2616|
+----------------+--------------------+----------+-------------+--------------------+------------------+--------+---------+---------+
only showing top 2 rows



In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "forecast.sales_data")

delta_table.alias("old_data") \
    .merge(
        df1.selectExpr("*", 
              "year(OrderDate) as year", 
              "month(OrderDate) as month", 
              "day(OrderDate) as day").alias("new_data"),
        "old_data.SalesOrderNumber = new_data.SalesOrderNumber"
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

In [0]:
%sql
select * from forecast.sales_data where year=2020;

SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount,year,month,day
SO48529,1,2020-11-11,Olivia Flores,olivia58@adventure-works.com,"Mountain-200 Black, 42",1,2049.0982,163.9279,2020,11,11
SO48527,1,2020-11-11,Isaiah Peterson,isaiah4@adventure-works.com,"Mountain-200 Silver, 38",1,2071.4196,165.7136,2020,11,11
SO48533,1,2020-11-11,Lindsey Shen,lindsey2@adventure-works.com,"Road-650 Red, 48",1,782.99,62.6392,2020,11,11
SO48526,1,2020-11-11,Charles Thompson,charles19@adventure-works.com,"Road-250 Black, 48",1,2181.5625,174.525,2020,11,11
SO48532,1,2020-11-11,Bianca Gao,bianca11@adventure-works.com,"Mountain-200 Black, 46",1,2049.0982,163.9279,2020,11,11
SO48530,1,2020-11-11,Gregory Becker,gregory24@adventure-works.com,"Mountain-200 Silver, 38",1,2071.4196,165.7136,2020,11,11
SO48531,1,2020-11-11,Whitney Kov�r,whitney4@adventure-works.com,"Mountain-200 Black, 42",1,2049.0982,163.9279,2020,11,11
SO48522,1,2020-11-11,Erik Rubio,erik21@adventure-works.com,"Road-250 Red, 52",1,2443.35,195.468,2020,11,11
SO48524,1,2020-11-11,Kelvin Lal,kelvin5@adventure-works.com,"Road-250 Black, 52",1,2181.5625,174.525,2020,11,11
SO48525,1,2020-11-11,Wyatt Anderson,wyatt10@adventure-works.com,"Road-550-W Yellow, 42",1,1000.4375,80.035,2020,11,11
